In [14]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import sentencepiece as spm
import re
from tqdm import tqdm

torch.backends.cudnn.benchmark = True

In [5]:
jokes_file = 'jokes_clean.txt'
train_file = 'jokes_clean.txt'

with open(jokes_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

fikralar = []
current_fikra = []
for line in lines:
    if line.strip() == '':
        if current_fikra:
            fikralar.append(' '.join(current_fikra))
            current_fikra = []
    else:
        current_fikra.append(line.strip())
if current_fikra:
    fikralar.append(' '.join(current_fikra))

with open(train_file, 'w', encoding='utf-8') as f:
    for fikra in fikralar:
        f.write(fikra + '\n\n')

In [6]:
spm.SentencePieceTrainer.train(
    input=train_file,
    model_prefix='fikra_tokenizer',
    vocab_size=10000,
    character_coverage=0.9995,
    model_type='bpe'
)
sp = spm.SentencePieceProcessor(model_file='fikra_tokenizer.model')

In [7]:
def tokenize_dataset(dataset_file):
    with open(dataset_file, 'r', encoding='utf-8') as file:
        text = file.read().strip()
    jokes = re.split(r'\n\s*\n', text)
    jokes = [re.sub(r'\s+', ' ', joke).strip() for joke in jokes if joke.strip()]
    tokenized_jokes = [sp.encode(joke, out_type=int) for joke in jokes]
    return tokenized_jokes

train_tokenized = tokenize_dataset('jokes_clean.txt')
print("Tokenized joke sayısı:", len(train_tokenized))

Tokenized joke sayısı: 777


In [8]:
# Dataset Tanımı ve tokenizasyon
class JokesDataset(Dataset):
    def __init__(self, tokenized_jokes, max_len=50, pad_token=0):
        self.sequences = []
        self.max_len = max_len
        for tokens in tokenized_jokes:
            if len(tokens) < max_len + 1:
                tokens = tokens + [pad_token] * (max_len + 1 - len(tokens))
            for i in range(len(tokens) - max_len):
                input_seq = tokens[i:i+max_len]
                target_seq = tokens[i+1:i+max_len+1]
                self.sequences.append((input_seq, target_seq))

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        input_seq, target_seq = self.sequences[idx]
        return torch.tensor(input_seq, dtype=torch.long), torch.tensor(target_seq, dtype=torch.long)


In [9]:
# PositionalEncoding: max_len burada 5000, üretim sırasında kullanılacak
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    def forward(self, x):
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len]

# Model Tanımı: Eğitimde max_len=50 kullanılıyor.
class SimpleDecoderLLM(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=4, num_layers=2, max_len=50):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len)  # Eğitimde giriş uzunluğu 50
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers)
        self.fc_out = nn.Linear(d_model, vocab_size)
        self.d_model = d_model
        self.max_len = max_len

    def forward(self, tgt, memory):
        tgt = self.token_embedding(tgt) * math.sqrt(self.d_model)
        tgt = self.pos_encoder(tgt)
        tgt = tgt.transpose(0, 1)
        seq_len = tgt.size(0)
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(seq_len).to(tgt.device)
        output = self.transformer_decoder(tgt, memory, tgt_mask=tgt_mask)
        logits = self.fc_out(output)
        return logits.transpose(0, 1)


In [10]:
# Parametreler
vocab_size = 10000
train_max_len = 50            # Eğitimde kullanılan maksimum giriş uzunluğu
gen_max_length = 150       # Üretim çıktısı olarak maksimum token sayısı
batch_size = 64
epochs = 100

# Dataset ve DataLoader
train_dataset = JokesDataset(train_tokenized, max_len=train_max_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(f"✅ GPU Kullanılıyor: {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU kullanılmıyor, CPU'ya geçildi!")

# Model oluşturulması (eğitimde kullanılan max_len = train_max_len)
model = SimpleDecoderLLM(vocab_size=vocab_size, d_model=256, nhead=4, num_layers=2, max_len=train_max_len)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

scaler = torch.amp.GradScaler('cuda')

✅ GPU Kullanılıyor: NVIDIA GeForce RTX 4060 Laptop GPU


In [26]:
train_losses = []
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch_inputs, batch_targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
        batch_inputs  = batch_inputs.to(device, non_blocking=True)
        batch_targets = batch_targets.to(device, non_blocking=True)
        optimizer.zero_grad()
        dummy_memory = torch.zeros(1, batch_inputs.size(0), model.d_model, device=device)
        with torch.amp.autocast(device_type='cuda'):
            outputs = model(batch_inputs, dummy_memory)
            outputs = outputs.contiguous().view(-1, vocab_size)
            batch_targets = batch_targets.contiguous().view(-1)
            loss = criterion(outputs, batch_targets)
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    scheduler.step()
    train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | LR: {scheduler.get_last_lr()[0]:.6f}")
print("Eğitim tamamlandı!")

Epoch 1/100 | Loss: 3.8340 | LR: 0.001000


Epoch 2/100 | Loss: 1.1762 | LR: 0.000999


Epoch 3/100 | Loss: 0.6422 | LR: 0.000998


Epoch 4/100 | Loss: 0.4695 | LR: 0.000996


Epoch 5/100 | Loss: 0.3876 | LR: 0.000994


...

Epoch 96/100 | Loss: 0.0679 | LR: 0.000004


Epoch 97/100 | Loss: 0.0680 | LR: 0.000002


Epoch 98/100 | Loss: 0.0676 | LR: 0.000001


Epoch 99/100 | Loss: 0.0676 | LR: 0.000000


Epoch 100/100 | Loss: 0.0676 | LR: 0.000000
Eğitim tamamlandı!


In [27]:
torch.save(model.state_dict(), 'model_v10.pth')
print("Model 'model_v10.pth' olarak kaydedildi!")

Model 'model_v10.pth' olarak kaydedildi!


In [17]:
model = SimpleDecoderLLM(vocab_size=vocab_size, d_model=256, nhead=4, num_layers=2, max_len=train_max_len)
checkpoint = torch.load('model_v10.pth', weights_only=True)
if checkpoint['pos_encoder.pe'].shape != model.pos_encoder.pe.shape:
    checkpoint['pos_encoder.pe'] = model.pos_encoder.pe
model.load_state_dict(checkpoint)
model.to(device)
model.eval()
print("Model yüklendi ve kullanıma hazır")

Model yüklendi ve kullanıma hazır


In [19]:
def generate_text(model, sp, start_text, max_length=gen_max_length, temperature=1.0, top_p=0.9, min_gen_length=50):
    model.eval()
    with torch.no_grad():
        tokens = sp.encode(start_text, out_type=int)
        if len(tokens) > model.max_len:
            tokens = tokens[-model.max_len:]
        generated = tokens.copy()
        dummy_memory = torch.zeros(1, 1, model.d_model, device=device)
        for _ in range(max_length - len(tokens)):
            trimmed_input = torch.tensor(generated[-model.max_len:], dtype=torch.long, device=device).unsqueeze(0)
            outputs = model(trimmed_input, dummy_memory)
            next_token_logits = outputs[:, -1, :] / temperature

            # Nucleus (top-p) sampling:
            sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True)
            cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
            sorted_indices_to_remove = cumulative_probs > top_p
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0
            next_token_logits[0, sorted_indices[sorted_indices_to_remove]] = -float('Inf')
            probs = torch.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1).item()
            generated.append(next_token)
            decoded = sp.decode(generated)
            if next_token == sp.eos_id():
                break
            # Eğer minimum token sayısına ulaşılmışsa ve çıktı son karakteri nokta, ünlem veya soru işareti ile bitiyorsa dur.
            if len(generated) >= min_gen_length and decoded.strip()[-1] in ['.', '!', '?']:
                break
        generated_text = sp.decode(generated)
        return generated_text

# Örnek kullanım:
start_text = "ensar hoca"  # Başlangıç metni
generated_fikra = generate_text(model, sp, start_text, max_length=gen_max_length, temperature=0.7, top_p=0.9, min_gen_length=50)
print("Üretilen Fıkra:", generated_fikra)

Üretilen Fıkra: ensar hoca bir gün pazarda dolaşırken bir adamın sürekli bedava akıl bedava akıl diye bağırdığını duymuş hoca yaklaşmış ve adama sormuş ne yapıyorsun böyle adam cevap vermiş insanlara bedava akıl dağıtıyorum bunun üzerine hoca o halde bana iki kilo ver demiş adam şaşırarak ama hoca akıl tartılmaz ki hoca gülümseyerek o zaman sen de bağırarak satma demiş adam şaşırarak ama hoca gülümseyerek o zaman sen de bağırarak satma demiş ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  tane don.


In [20]:
with open("jokes_clean.txt", "r", encoding="utf-8") as fin, open("jokes_preprocessed.txt", "w", encoding="utf-8") as fout:
    joke = []
    for line in fin:
        if line.strip() == "":
            if joke:
                fout.write(" ".join(joke).strip() + " <EOS>\n")
                joke = []
        else:
            joke.append(line.strip())
    if joke:
        fout.write(" ".join(joke).strip() + " <EOS>\n")
